In [1]:
import pandas as pd

# 讀取CSV檔案
data = pd.read_csv("train.csv")

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 處理缺失值: 使用中位數填充
data.fillna(data.median(), inplace=True)

# 將入職日期轉換為距今的天數
data['Days Since Joining'] = (pd.Timestamp.now() - pd.to_datetime(data['Date of Joining'])).dt.days
data.drop('Date of Joining', axis=1, inplace=True)

# 使用LabelEncoder對類別變數進行編碼
label_encoders = {}
categorical_columns = ['Gender', 'Company Type', 'WFH Setup Available']

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# 切分數據為訓練集和驗證集
X = data.drop(['Employee ID', 'Burn Rate'], axis=1)
y = data['Burn Rate']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# 標準化特徵
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

X_train[:5], y_train.head()

<ipython-input-2-ea1df4843c65>:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.fillna(data.median(), inplace=True)


(array([[ 1.05153657,  0.72701061, -1.0763685 , -0.16394616,  0.26559799,
          0.52066695, -0.40931329],
        [ 1.05153657,  0.72701061,  0.92904986,  0.71631191,  0.26559799,
          0.07916442, -1.47837076],
        [ 1.05153657,  0.72701061, -1.0763685 ,  0.71631191,  0.7711595 ,
          0.6862304 ,  1.21792461],
        [-0.95098927,  0.72701061, -1.0763685 , -0.16394616, -0.23996353,
         -1.68684572, -0.35254918],
        [-0.95098927,  0.72701061, -1.0763685 , -0.16394616,  0.26559799,
          0.13435223, -0.87288688]]),
 8387    0.51
 1817    0.45
 4386    0.62
 9317    0.45
 4575    0.45
 Name: Burn Rate, dtype: float64)

In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 定義模型結構
model = keras.Sequential([
    layers.InputLayer(input_shape=(X_train.shape[1],)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='linear')  # 使用linear激活函數，因為這是一個回歸問題
])

# 編譯模型
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

# 訓練模型
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_valid, y_valid), verbose=1)


Epoch 1/100
253/253 [==============================] - 7s 5ms/step - loss: 0.0244 - mae: 0.1170 - val_loss: 0.0176 - val_mae: 0.1083
Epoch 2/100
253/253 [==============================] - 1s 4ms/step - loss: 0.0111 - mae: 0.0808 - val_loss: 0.0113 - val_mae: 0.0806
Epoch 3/100
253/253 [==============================] - 1s 4ms/step - loss: 0.0090 - mae: 0.0718 - val_loss: 0.0107 - val_mae: 0.0787
Epoch 4/100
253/253 [==============================] - 1s 4ms/step - loss: 0.0084 - mae: 0.0687 - val_loss: 0.0117 - val_mae: 0.0841
Epoch 5/100
253/253 [==============================] - 2s 6ms/step - loss: 0.0080 - mae: 0.0667 - val_loss: 0.0095 - val_mae: 0.0753
Epoch 6/100
253/253 [==============================] - 1s 6ms/step - loss: 0.0075 - mae: 0.0648 - val_loss: 0.0115 - val_mae: 0.0833
Epoch 7/100
253/253 [==============================] - 1s 4ms/step - loss: 0.0073 - mae: 0.0640 - val_loss: 0.0128 - val_mae: 0.0874
Epoch 8/100
253/253 [==============================] - 1s 4ms/step - 

In [6]:
# 讀取test.csv
test_data = pd.read_csv("test.csv")

# 處理缺失值: 使用中位數填充
test_data.fillna(data.median(), inplace=True)

# 將入職日期轉換為距今的天數
test_data['Days Since Joining'] = (pd.Timestamp.now() - pd.to_datetime(test_data['Date of Joining'])).dt.days
test_data.drop('Date of Joining', axis=1, inplace=True)

# 使用之前的LabelEncoder對類別變數進行編碼
for col in categorical_columns:
    test_data[col] = label_encoders[col].transform(test_data[col])

# 移除Employee ID並標準化特徵
X_test = test_data.drop('Employee ID', axis=1)
X_test = scaler.transform(X_test)

X_test[:5]

array([[ 1.05153657,  0.72701061,  0.92904986, -1.92446229, -1.75664808,
        -3.17691677, -1.27023567],
       [ 1.05153657, -1.37549575, -1.0763685 ,  0.71631191,  1.78228253,
         1.18292075,  1.05709296],
       [ 1.05153657,  0.72701061, -1.0763685 ,  1.59656997,  2.28784405,
         1.29329639,  0.83949719],
       [ 1.05153657, -1.37549575, -1.0763685 , -1.04420423, -0.74552504,
        -0.03121122, -0.87288688],
       [-0.95098927,  0.72701061, -1.0763685 ,  0.71631191,  0.26559799,
         0.07916442,  1.6814982 ]])

In [7]:
predictions = model.predict(X_test)

62/62 [==============================] - 0s 2ms/step


In [8]:
df_ans = pd.DataFrame(predictions.astype(float), columns=['Burn Rate'])
df_ans.to_csv('mySubmission.csv', index_label='Employee ID')